# SUSI sweRun
This script runs Susi* for one or more than one sites in Sweden.
It tryes to facilitate a form to get the parameters, collects the smhi weather data and storage the outputs in a centralized folder per site.

Follow the link to susi ....
get the general instructions from ...

Alejandro G.
 /// I need to improve the description ///

## Setting up

In [74]:
# Importing general libraries

import numpy as np
import pandas as pd
from datetime import datetime
import configparser
import argparse
import matplotlib.pylab as plt
import seaborn as sns
import sys
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from scipy.misc import derivative
from scipy.interpolate import InterpolatedUnivariateSpline as interS
from scipy.sparse import  diags
from scipy.sparse.linalg import  spsolve
#----------------------------------------------------------------------------
# Install non-standard packages
# !pip install xlsxwriter
# !pip install xlrd==1.2.0
# !pip install pyproj
# #--------------------------
# !pip install netCDF4
# !pip install datetime
# !pip install xlrd --upgrade
# !pip install dataframe_image
# # --------------------------
from pyproj import CRS, Transformer
import os

In [75]:
# Importing repositories 
if not os.path.exists('susi_2022'): 
    !git clone https://github.com/annamarilauren/susi_2022
else: print("Susi repository is already in yor computer")

if not os.path.exists('susi_SMHI'): 
    !git clone https://github.com/alejandrogandara/susi_SMHI
else: print("SMHI collect repository is already in yor computer")

Susi repository is already in yor computer
SMHI collect repository is already in yor computer


In [76]:
# Importing local libraries

import scripts.import_param as importParam # script to import from xls to dict
import susi_SMHI.susi_SMHI as smhi

newpathsusi = 'susi_2022'
if newpathsusi not in sys.path: sys.path.insert(0,newpathsusi)
newpathsusi = 'susi_SMHI/'
if newpathsusi not in sys.path: sys.path.insert(0,newpathsusi)
# -----------------------------------------------------------------------------
from susi.susi_utils import read_FMI_weather
from susi.susi_main import Susi
from inputs.susi_para import get_susi_para
from susi.susi_io import *

print (f"SUSI is ready to run!")

SUSI is ready to run!


### Local functions

In [77]:
# gets parameters from  file
# This script provides functionality to get data from an xmlx file. You will find a template at ./inputs/param_template.xlsx
# pFile: parameter file

from pandas import read_excel
import traceback
from datetime import datetime

def getParamFile(path):
    try: pFile = read_excel(path)
    except: 
        print('file does not exists')
        return
    return pFile

def getVariablesFromFile(pFile, site):
    # var [True, False]
    # file ['file path']
    vars = {}
    pFile=pFile[(pFile['commented'] != '#') & (pFile['path'].isna())]
    pFile.reset_index()
    for ind in pFile.index:
        if pFile['type'][ind] == 'date': 
            
            date = datetime.strptime(pFile[site][ind], '%Y-%m-%d %H:%M:%S')
            globals()[pFile['var'][ind]] = date
            vars[pFile['var'][ind]] = date
            #datetime.datetime(date.year,date.month,date.day)
        else: 
            globals()[pFile['var'][ind]] = pFile[site][ind] 
            vars[pFile['var'][ind]] = pFile[site][ind]
        print('variable updated: ' + pFile['var'][ind]+' = '+ str(pFile[site][ind]))
    return vars

# gets parameters from param file and fill dictionaries in memory
def getDicInFile(pFile, site):
    pFile=pFile[(pFile['commented'] != '#') & (pFile['path'].notnull())]
    pFile.reset_index()

    for uvar in pFile['var'].unique():
        sp = pFile[pFile['var'] == uvar]
        print('------------------------------')
        print('dictionary: '+ uvar + '  keys:' + str(len(sp)))
        globals()[uvar]
        for ind in sp.index:
            try: 
                path = pFile['path'][ind].replace(', ','][')
                if pFile['type'][ind] == 'str': 
                    value = "'"+ str(pFile[site][ind]) + "'"
                else: value = str(pFile[site][ind])
                #exec(pFile['var'][ind] + pFile['path'][ind].replace(',','][').replace(' ', '') + ' = ' + str(pFile[site][ind]))
                line = uvar + path + ' = ' + value
                print(line)
                exec(line)
            #except: print('ERROR: '+ uvar + path + ' = ' + value + ' NOT UPDATED' )
            except Exception: traceback.print_exc()

### Reading the parameter file, setting up the folders; motti and weather files.

In [78]:
# Name convention
# weather file:     inputs/.../weather/  {site}_weather.csv
# motti file:       inputs/.../motti/    {site}_motti_lyr_0.xlsx
# You will define the input folder later

folderName= importParam.mkfolder('outputs/')               # output folder
graph_folder = importParam.mkfolder(f'{folderName}/graphs/')

weatherPath = 'inputs/sweden/weather/'                          # Where the weather file is
#mottipath = 'inputs/sweden/motti/'                         # folder where motti files are
mottipath = 'inputs/sweden/heureka/'                         # folder where motti files are


paramFile = getParamFile("inputs\sweden\parameters.xlsx")  # map the param file

sitesNames = paramFile.columns[9:]  # extract the site names from param file header

# Defines motti files, please follow the name convention or add it manually
data_files = {}
for key in sitesNames:
    data_files[key] = {}
    data_files[key]['para'] = f'susi_para_{key}'
    data_files[key]['weatherFile'] = f'{key}_weather.csv'
    data_files[key]['motti_lyr_0'] = f'{key}_heureka_input_lyr_0.xlsx'
    data_files[key]['motti_lyr_1'] = f'{key}_heureka_input_lyr_0.xlsx'        # fake, just to fill the gap
    data_files[key]['motti_lyr_2'] = f'{key}_heureka_input_lyr_0.xlsx'        # fake, just to fill the gap


outputs/ already exists, ready to go
outputs//graphs/ already exists, ready to go


## Which site are you running?

In [79]:
print(pd.DataFrame(sitesNames))

                  0
0         01_Bjdamm
1         03_Korpis
2       04_Fröjered
3        05_SträRed
4         06_Risabo
5        07_Rothult
6        08_Ydreham
7        09_Gröngöl
8      10_Hällehult
9         11_Ökalix
10        12_Ökalix
11        13_Ökalix
12       14_Höglund
13      15_Jörnmark
14  Norunda-Uppsala
15      Hälsingfors
16    Strömsjöliden
17         Skogaryd
18           Degerö
19         Krycklan
20      Trollberget


In [80]:
#wsite = sites[ind]
#wsite

In [81]:
siteID = 20
site = list(data_files.keys())[siteID]
site

'Trollberget'

In [82]:
#for siteID in range(14, 21):
#Get the weather 
    #print(f'{site} from {start_date} to {end_date}')
    #site = list(data_files.keys())[siteID]
    #workingFolder = os.getcwd()
    #wfile = smhi.getWeather(site, wpara['lon'], wpara['lat'], start_date, end_date, weatherPath, workingFolder, stations_nearby= 10)
#    print( wpara['lon'])

### 3.1 Default inputs

In [83]:
mottifile = {'path':mottipath,
              'dominant':{1: data_files[site]['heureka_lyr_0']},
              'subdominant':{0:data_files[site]['heureka_lyr_1']},
              'under':{0:data_files[site]['heureka_lyr_2']}}

weatherData = data_files[site]['weatherFile']
sarkaSim = paramFile.loc[paramFile.path == "['start_date']"][site].values[0]
start_date = paramFile.loc[paramFile.path == "['start_date']"][site].values[0]
end_date = paramFile.loc[paramFile.path == "['end_date']"][site].values[0]
start_yr = start_date.year 
end_yr = end_date.year
yrs = (end_date - start_date).days/365.25

sarkaSim = 40.                                                                  # strip width, ie distance between ditches, m
n = int(sarkaSim / 2)                                                           # number of computation nodes in the strip


#age = paramFile.loc[paramFile.path == "['age']"][site].values[0]
dominant_age = float(paramFile.loc[paramFile['var'] == "age_dominant"][site].values[0])
#dominant_age = 85.

ageSim = {'dominant': dominant_age * np.ones(n),
          'subdominant': 0*np.ones(n),
          'under': 0*np.ones(n)}                                                         # age of the stand in each node

sfc =  np.ones(n, dtype=int)*3                                                            # site fertility class


#ageSim['dominant'][int(n/2):] = 2.
#ageSim[4:-4] = 2.

          
wpara, cpara, org_para, spara, outpara, photopara = get_susi_para(wlocation='undefined', peat='krycklan', 
                                                                          folderName=folderName, hdomSim=None,  
                                                                          ageSim=ageSim, sarkaSim=sarkaSim, sfc=sfc, susiPath = weatherPath,
                                                                          n=n)

#spara['canopylayers']['dominant'][int(n/2):] = 2                                                                        
#spara['canopylayers']['subdominant'][:int(n/2)] = 1                                                                        

#Get the parameters to dictionary
getDicInFile(paramFile, site)



------------------------------
dictionary: wpara  keys:9
wpara['start_date'] = '2020-05-01 00:00:00'
wpara['end_date'] = '2020-10-31 00:00:00'
wpara['infolder'] = 'nan'
wpara['infile_d'] = 'nan'
wpara['start_yr'] = 2020
wpara['end_yr'] = 2020
wpara['description'] = 'nan'
wpara['lat'] = 64.1765679074559
wpara['lon'] = 19.862037007013
------------------------------
dictionary: spara  keys:16
spara['sitename'] = 'nan'
spara['species'] = 'nan'
spara['hdom'] = nan
spara['vol'] = nan
spara['smc'] = 'nan'
spara['nLyrs'] = nan
spara['dzLyr'] = nan
spara['ditch depth west'] = nan
spara['ditch depth east'] = nan
spara['ditch depth 20y west'] = nan
spara['ditch depth 20y east'] = nan
spara['scenario name'] = nan
spara['slope'] = nan
spara['peat type'] = nan
spara['peat type bottom'] = nan
spara['vonP'] = nan
------------------------------
dictionary: outpara  keys:1
outpara['netcdf'] = 'nan'


Traceback (most recent call last):
  File "C:\Users\aoga0001\AppData\Local\Temp\3\ipykernel_20600\2725772339.py", line 54, in getDicInFile
    exec(line)
  File "<string>", line 1, in <module>
NameError: name 'nan' is not defined
Traceback (most recent call last):
  File "C:\Users\aoga0001\AppData\Local\Temp\3\ipykernel_20600\2725772339.py", line 54, in getDicInFile
    exec(line)
  File "<string>", line 1, in <module>
NameError: name 'nan' is not defined
Traceback (most recent call last):
  File "C:\Users\aoga0001\AppData\Local\Temp\3\ipykernel_20600\2725772339.py", line 54, in getDicInFile
    exec(line)
  File "<string>", line 1, in <module>
NameError: name 'nan' is not defined
Traceback (most recent call last):
  File "C:\Users\aoga0001\AppData\Local\Temp\3\ipykernel_20600\2725772339.py", line 54, in getDicInFile
    exec(line)
  File "<string>", line 1, in <module>
NameError: name 'nan' is not defined
Traceback (most recent call last):
  File "C:\Users\aoga0001\AppData\Local\Temp\

### Getting weather data

In [84]:
#Get the weather 
workingFolder = os.getcwd()
wfile = smhi.getWeather(site, wpara['lon'], wpara['lat'], start_date, end_date, weatherPath, workingFolder, stations_nearby= 10)

t_mean: 1 t_max: 1 t_min: 1 rainfall: 1 radiation: 1 humidity: 4 

             varType  percent_missing resampled   nStations         site
varType                                                                 
t_mean        t_mean              0.0     False           1  Trollberget
t_max          t_max              0.0     False           1  Trollberget
t_min          t_min              0.0     False           1  Trollberget
rainfall    rainfall              0.0     False           1  Trollberget
radiation  radiation              0.0     False           1  Trollberget
humidity    humidity              0.0     False           4  Trollberget
hpa              NaN              NaN       NaN  calculated  Trollberget

 site Trollberget: 	OK -- from 2020-05-01 to 2020-10-31 -- (184 days, 1 years) max 10 stations nearby


In [85]:
bgvf

NameError: name 'bgvf' is not defined

In [ ]:
site, wpara['lon'], wpara['lat'], start_date, end_date, weatherPath, workingFolder

### Importing parameters from table

In [ ]:
# Print some of this
#print(f" {spara['ditch depth west']} \n {outpara['netcdf']} \n {spara} \n {weatherData} ")


### Checking the weather data
The data is collected from SMHI using the dates and coordinates from the param file, but is possible to use the susi_SMHI module manually.


In [ ]:
print(workingFolder)

# read weather input
forc=read_FMI_weather(0, start_date, end_date, sourcefile=wfile)

### 3.4 Run the simulation

In [ ]:
# in case Susi() returns write permission error
#ncf.close()

## Running SUSI

In [ ]:
from susi.susi_io import *
graph_output = graph_folder + weatherData[:-4] + '_graph.png'
weather_fig(forc)        #Draw the weather input figure

In [ ]:
print("RUNING SUSI")
susi = Susi()
susi.run_susi(forc, wpara, cpara, org_para, spara, outpara, photopara, start_yr, end_yr, wlocation = 'undefined', 
                                mottifile=mottifile, peat= 'krycklan', photosite='All data', 
                                folderName=folderName,ageSim=ageSim, sarkaSim=sarkaSim, sfc=sfc)

## Drawing figures

In [ ]:
from susi.figures import *
from netCDF4 import Dataset 

#ff = r'susi_2022/outputs/susi.nc'
ff=outpara['outfolder']+ outpara['netcdf']  # AG added
#ff=r'outputs/swe_site_05_StraRed_susi.nc'
scen = 0

hydrology(ff, scen)
stand(ff, scen)
mass(ff, scen)
carbon(ff, scen)
compare_1(ff, [0,1])


In [ ]:
compare_scens(ff)

In [ ]:
# v_ini, v, iv5,  cbt, dcbt, cb, dcb, w,dw,logs,pulp, dv,dlogs,dpulp,yrs, bmgr,  \
#                                     Nleach, Pleach, Kleach, DOCleach, runoff, nrelease, \
#                                     prelease, krelease, ch4release = run_susi(forc, wpara, cpara, 
#                                     org_para, spara, outpara, photopara, start_yr, end_yr, wlocation = 'undefined', 
#                                     mottifile=mottifile, peat= 'other', photosite='All data', 
#                                     folderName=folderName,ageSim=ageSim, sarkaSim=sarkaSim, sfc=sfc, susiPath=susiPath) 